In [ ]:
!pip install -q accelerate peft bitsandbytes transformers trl
!pip install -q numpy
!pip install -q pandas
!pip install -q evaluate


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.6/85.6 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 101.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.0/118.0 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 72.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 103.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 43.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForSequenceClassification,
    Trainer,
    DataCollatorWithPadding,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
# from peft import LoraConfig, PeftModel
from peft import LoraConfig, TaskType, get_peft_model, prepare_model_for_kbit_training, PeftModel, AutoPeftModelForCausalLM

from trl import SFTTrainer
import json

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, precision_recall_fscore_support

import numpy as np

import pandas as pd
import ast

import evaluate
from functools import partial

import bitsandbytes as bnb





In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!huggingface-cli login

Mounted at /content/drive

    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
def get_file_data(file):
  with open(file, 'rt') as f:
    data = json.load(f)

  return data

In [ ]:
# The model that you want to train from the Hugging Face hub
model_name = "meta-llama/Llama-2-7b-hf"
# model_name = "togethercomputer/LLaMA-2-7B-32K"

# The instruction dataset to use
# dataset_name = "mlabonne/guanaco-llama2-1k"

# Fine-tuned model name
new_model = "llama-2-7b-apk-features-ft-2"

################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 64

# Alpha parameter for LoRA scaling
lora_alpha = 16

# Dropout probability for LoRA layers
lora_dropout = 0.1

################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"

# Number of training epochs
num_train_epochs = 1

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = False

# Batch size per GPU for training
per_device_train_batch_size = 4

# Batch size per GPU for evaluation
per_device_eval_batch_size = 4

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule (constant a bit better than cosine)
lr_scheduler_type = "constant"

# Number of training steps (overrides num_train_epochs)
max_steps = -1

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 25

# Log every X updates steps
logging_steps = 5

################################################################################
# SFT parameters
################################################################################

# Maximum sequence length to use
max_seq_length = None

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = {"": 0}

In [ ]:
def partition_dataset(df):
  # Let's say we want to split the data in 80:10:10 for train:valid:test dataset
  # First we split 80:20 train:remainder
  # Then we split the remainder by 0.5
  train_size=0.90
  test_size=0.50

  df_sorted = df.sort_values(by='dex_date', ascending=True)

  # In the first step we will split the data in training and remaining dataset
  df_train, df_rem = train_test_split(df_sorted, shuffle=False, train_size=train_size)

  # Now since we want the valid and test size to be equal (10% each of overall data).
  # we have to define valid_size=0.5 (that is 50% of remaining data)

  df_val, df_test = train_test_split(df_rem, shuffle=False, test_size=test_size)

  df_train.to_csv("train.csv", index=False, encoding="utf-8")
  df_val.to_csv("val.csv", index=False, encoding="utf-8")
  df_test.to_csv("test.csv", index=False, encoding="utf-8")


In [ ]:
def maintain_class_ratio(dataset, lower_class_ratio=0.1, upper_class_ratio=0.13):
  for split in dataset:
    total_items = dataset[split].num_rows
    label_1_ratio = np.array(dataset_split[split]['label']).sum()/len(dataset_split[split]['label'])
    total_label_1 = np.array(dataset_split[split]['label']).sum()
    if(label_1_ratio < lower_class_ratio):
      print('Not enough malware')

      # Calculate new total to achieve 8% label 1
      new_total = total_label_1 / lower_class_ratio

      # Calculate number of items to remove
      items_to_remove = total_items - new_total
      items_to_remove = round(items_to_remove)  # rounding since we can't remove partial items

      print(f'Removing {items_to_remove} benign samples to maintain class ratio')

      # Let's say we want to remove last 5 items where the 'label' is a particular value, say 1.
      matching_indices = [i for i, item in enumerate(dataset[split]) if item['label'] == 0]

      # Get the last 5 indices
      indices_to_remove = matching_indices[-(items_to_remove):]

      # Filter the dataset
      filtered_dataset = dataset[split].filter(lambda example, idx: idx not in indices_to_remove, with_indices=True)
      dataset[split] = filtered_dataset

    elif(label_1_ratio > upper_class_ratio):
      print('Too much malware')
      desired_label_1_items = round(total_items * upper_class_ratio)

      items_to_remove = total_label_1 - desired_label_1_items

      print(f'Removing {items_to_remove} malware samples to maintain class ratio')

      # Let's say we want to remove last 5 items where the 'label' is a particular value, say 0.
      matching_indices = [i for i, item in enumerate(dataset[split]) if item['label'] == 1]

      # Get the last 5 indices
      indices_to_remove = matching_indices[-(items_to_remove):]

      # Filter the dataset
      filtered_dataset = dataset[split].filter(lambda example, idx: idx not in indices_to_remove, with_indices=True)
      dataset[split] = filtered_dataset

    else:
      print('Within class ratio')
      dataset[split] = dataset[split]

  return dataset



In [ ]:
def preprocess_batch(batch, tokenizer, max_length):
    """
    Tokenizes dataset batch

    :param batch: Dataset batch
    :param tokenizer: Model tokenizer
    :param max_length: Maximum number of tokens to emit from the tokenizer
    """

    return tokenizer(
        batch["text"],
        max_length = max_length,
        truncation = True,
        padding="max_length",
    )

# SOURCE https://github.com/databrickslabs/dolly/blob/master/training/trainer.py
def preprocess_dataset(tokenizer: AutoTokenizer, max_length: int, seed, dataset: str):
    """
    Tokenizes dataset for fine-tuning

    :param tokenizer (AutoTokenizer): Model tokenizer
    :param max_length (int): Maximum number of tokens to emit from the tokenizer
    :param seed: Random seed for reproducibility
    :param dataset (str): Instruction dataset
    """

    # Add prompt to each sample
    print("Preprocessing dataset...")
    # dataset = dataset.map(create_prompt_formats)

    # print(dataset)

    # Apply preprocessing to each batch of the dataset & and remove "instruction", "input", "output", and "text" fields
    _preprocessing_function = partial(preprocess_batch, max_length = max_length, tokenizer = tokenizer)
    dataset_tokenized = dataset.map(
        _preprocessing_function,
        batched = True,
        remove_columns = ['dex_date', 'family', 'class', 'filename', 'permissions', 'num_activities', 'num_services', 'num_files', 'certificates', 'embedded_urls', 'api_calls', 'intents', 'token_size', 'text']
    )

    # dataset_tokenized = dataset.map(preprocess_batch, batched=True, fn_kwargs={"tokenizer": tokenizer, "max_length": max_length})

    # Filter out samples that have "input_ids" exceeding "max_length"
    dataset_tokenized = dataset_tokenized.filter(lambda sample: len(sample["input_ids"]) <= max_length)

    # Shuffle dataset
    dataset_tokenized = dataset_tokenized.shuffle(seed = seed)

    return dataset_tokenized

In [ ]:
### Instruction: <instruction>\n\n### Context: <context>\n\n### Response: <response>\n\n### End

def format_prompt(sample, create_response=True):
  """
  Creates a formatted prompt template for a prompt in the instruction dataset

  :param sample: Prompt or sample from the instruction dataset
  """
  # Initialize static strings for the prompt template
  INSTRUCTION_KEY = "### Instruction:"
  INTRO_BLURB = "Using the following features extracted from an android app, reply only with YES or NO if the app is malicious."
  CONTEXT_KEY = "### Context:"
  # INSTRUCTION_KEY = "```"
  INPUT_KEY = "Input:"
  RESPONSE_KEY = "### Response:"
  END_KEY = "```"

  PROMPT = """
    {instruction_key} {intro}
    {context_key}
    Permissions: {permissions}
    Intents: {intents}
    Activities: {activities}
    Services: {services}
    API Calls: {api_calls}
    Embedded Urls: {urls}
    Number of Files: {files}
    Certificates:
    Summary: {cert_summary}
    Certs: {certs}
    {response_key} {response}
  """.format(
  intro=INTRO_BLURB,
  instruction_key=INSTRUCTION_KEY,
  context_key=CONTEXT_KEY,
  permissions="{permissions}",
  intents="{intents}",
  activities="{activities}",
  services="{services}",
  api_calls="{api_calls}",
  urls="{urls}",
  files="{files}",
  cert_summary="{cert_summary}",
  certs="{certs}",

  # instruction="{instruction}",
  # input_key=INPUT_KEY,
  # input="{input}",
  response_key=RESPONSE_KEY,
  response="{response}",
  end_key=END_KEY
  )


  # permissions = ", ".join(ast.literal_eval(sample["permissions"]))
  permissions = sample["permissions"]
  intents = sample["intents"]
  activities = sample["num_activities"]
  services = sample["num_services"]
  api_calls = sample["api_calls"]
  urls = sample["embedded_urls"]
  files = sample["num_files"]


  cert_info = ast.literal_eval(sample["certificates"])
  # cert_info = sample["certificates"]
  cert_summary = cert_info['cert_summary']
  certs = cert_info['certs']

  if create_response:
    response = 'YES' if sample['label'] == 1 else 'NO'
  else:
    response = ''

  full_prompt = PROMPT.format(permissions=permissions, intents=intents, activities=activities, services=services, api_calls=api_calls, urls=urls, files=files,cert_summary=cert_summary, certs=certs, response=response)

  return { "text": full_prompt, "label": int(sample['label'])}

In [ ]:
def format_prompt_no_response(sample):
  return format_prompt(sample, create_response=False)

In [ ]:
json_data = get_file_data(f'/content/drive/MyDrive/fine-tuning-data/apk_features_concat_new.json')
# json_data = get_file_data(f'/content/drive/MyDrive/fine-tuning-data/batch-1.json')

json_data = json_data[:5100]
# json_data = json_data[:50000]
# remove elements with no features
json_data = [i for i in json_data if i is not None]

# df = pd.DataFrame(json_data[:20])
df = pd.DataFrame(json_data)

del json_data

# only use token size less than 3000
df = df[df['token_size'] < 4000]

#convert date
df['dex_date'] = pd.to_datetime(df['dex_date'], format='%Y-%m-%d %H:%M:%S')

# dataset = Dataset.from_pandas(df)

# dataset = dataset.map(create_prompt_formats)

# partition_dataset(dataset.to_pandas())
partition_dataset(df)


dataset_split = load_dataset(
    "csv",
    data_files={
        "train": "train.csv",
        "test": "test.csv",
        "validation": "val.csv",
    },
)

dataset_split["train"] = dataset_split["train"].map(format_prompt)
dataset_split["validation"] = dataset_split["validation"].map(format_prompt)
dataset_split["test"] = dataset_split["test"].map(format_prompt_no_response)

print(dataset_split)

# print(type(dataset_split['train'][0]['dex_date']))

dataset_split_with_ratio = maintain_class_ratio(dataset_split)


# display % of training data with label=1
print(np.array(dataset_split_with_ratio['train']['label']).sum()/len(dataset_split_with_ratio['train']['label']))
print(np.array(dataset_split_with_ratio['test']['label']).sum()/len(dataset_split_with_ratio['test']['label']))
print(np.array(dataset_split_with_ratio['validation']['label']).sum()/len(dataset_split_with_ratio['validation']['label']))

print(dataset_split_with_ratio)




Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/4529 [00:00<?, ? examples/s]

...

Not enough malware
Removing 62 benign samples to maintain class ratio


Filter:   0%|          | 0/252 [00:00<?, ? examples/s]

Not enough malware
Removing 102 benign samples to maintain class ratio


Filter:   0%|          | 0/252 [00:00<?, ? examples/s]

0.1
0.1
0.1
DatasetDict({
    train: Dataset({
        features: ['label', 'dex_date', 'family', 'class', 'filename', 'permissions', 'num_activities', 'num_services', 'num_files', 'certificates', 'embedded_urls', 'api_calls', 'intents', 'token_size', 'text'],
        num_rows: 4260
    })
    test: Dataset({
        features: ['label', 'dex_date', 'family', 'class', 'filename', 'permissions', 'num_activities', 'num_services', 'num_files', 'certificates', 'embedded_urls', 'api_calls', 'intents', 'token_size', 'text'],
        num_rows: 190
    })
    validation: Dataset({
        features: ['label', 'dex_date', 'family', 'class', 'filename', 'permissions', 'num_activities', 'num_services', 'num_files', 'certificates', 'embedded_urls', 'api_calls', 'intents', 'token_size', 'text'],
        num_rows: 150
    })
})


In [ ]:
print(dataset_split_with_ratio["test"][4]["text"])
print(dataset_split_with_ratio["test"][5]["text"])
print(dataset_split_with_ratio["test"][6]["text"])
print(dataset_split_with_ratio["test"][7]["text"])
print(dataset_split_with_ratio["test"][8]["text"])


    ### Instruction: Using the following features extracted from an android app, reply only with YES or NO if the app is malicious.
    ### Context:
    Permissions: ['INTERNET: have full network access', 'ACCESS_NETWORK_STATE: view network connections']
    Intents: [['android.intent.action.MAIN']]
    Activities: 5
    Services: 0
    API Calls: ['getState', 'substring', 'append', 'close', 'valueOf', 'myPid', 'loadClass', 'addFlags', 'delete', 'cancel', 'getInstance', 'init', 'obtainMessage', 'setFlags', 'getApplicationInfo', 'getDeviceId', 'indexOf', 'openFileOutput', 'update', 'putExtra', 'query', 'setDataAndType', 'replaceAll', 'insert', 'getBytes', 'write', 'sendMessage', 'doFinal', 'getOutputStream', 'getInputStream', 'exists', 'read', 'getNetworkOperator', 'setRequestMethod', 'startService']
    Embedded Urls: ['http://schemas.android.com/apk/res/android', 'https://csi.gstatic.com/csi']
    Number of Files: 378
    Certificates:
    Summary: APK is signed with v1
    Certs: ['

In [ ]:
def find_all_linear_names(model):
    """
    Find modules to apply LoRA to.

    :param model: PEFT model
    """

    cls = bnb.nn.Linear4bit
    lora_module_names = set()
    for name, module in model.named_modules():
        if isinstance(module, cls):
            names = name.split('.')
            lora_module_names.add(names[0] if len(names) == 1 else names[-1])

    if 'lm_head' in lora_module_names:
        lora_module_names.remove('lm_head')
    print(f"LoRA module names: {list(lora_module_names)}")
    return list(lora_module_names)

In [ ]:
# import accuracy evaluation metric
accuracy = evaluate.load("accuracy")

In [ ]:
# define an evaluation function to pass into trainer later
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=1)

    return {"accuracy": accuracy.compute(predictions=predictions, references=labels)}

In [ ]:
# Load dataset (you can process it here)
# dataset = load_dataset(dataset_name, split="train")

# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard",

    evaluation_strategy="steps",
    eval_steps=0.2,
)

# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset_split_with_ratio["train"],
    eval_dataset=dataset_split_with_ratio["validation"],
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

# Train model
trainer.train()

# Save trained model
trainer.model.save_pretrained(new_model)


Your GPU supports bfloat16: accelerate training with bf16=True


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:122: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:166: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/4260 [00:00<?, ? examples/s]

Map:   0%|          | 0/150 [00:00<?, ? examples/s]

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss
213,0.564100,0.471219


Step,Training Loss,Validation Loss
213,0.564100,0.471219
426,0.380700,0.393403
639,0.349900,0.367671
852,0.303100,0.360690
1065,0.284100,0.332305


In [ ]:
%load_ext tensorboard
%tensorboard --logdir results/runs

In [ ]:
# Reload model in FP16 and merge it with LoRA weights
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map,
)
model = PeftModel.from_pretrained(base_model, new_model)
model = model.merge_and_unload()

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
!huggingface-cli login



model.push_to_hub(new_model, use_temp_dir=False)
tokenizer.push_to_hub(new_model, use_temp_dir=False)


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
Your token has been saved to /roo

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/brendonduncan/llama-2-7b-apk-features-ft-2/commit/787d6fd6af1bff56b7701576a72415c533eba895', commit_message='Upload tokenizer', commit_description='', oid='787d6fd6af1bff56b7701576a72415c533eba895', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
new_model_hf = "brendonduncan/llama-2-7b-apk-features-ft-2"
# Get the tokenizer
tokenizer = AutoTokenizer.from_pretrained(new_model_hf)
# Load the model
model = AutoModelForCausalLM.from_pretrained(new_model_hf, load_in_4bit=True,
 torch_dtype=torch.float16,
 device_map=device_map)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
def generate_output(data):
  # print(data)
  prompt = data["text"]
  actual_output = data["label"]

  # Tokenize the input
  input_ids = tokenizer(prompt, return_tensors="pt", truncation=True).input_ids.cuda()

  # Run the model to infere an output
  # outputs = model.generate(input_ids=input_ids, max_new_tokens=100, do_sample=True, top_p=0.9,temperature=0.5)
  outputs = model.generate(input_ids=input_ids, max_new_tokens=100, do_sample=True, top_p=0.7,temperature=1)

  return {"prompt": prompt, "response": f'###Label: {actual_output}\n ###Prediction: {tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0][len(prompt):]}', "label": actual_output}

In [ ]:
all_results = []

dataset = dataset_split_with_ratio["test"]
all_results = dataset.map(generate_output)

print(all_results)


Map:   0%|          | 0/190 [00:00<?, ? examples/s]

Dataset({
    features: ['label', 'dex_date', 'family', 'class', 'filename', 'permissions', 'num_activities', 'num_services', 'num_files', 'certificates', 'embedded_urls', 'api_calls', 'intents', 'token_size', 'text', 'prompt', 'response'],
    num_rows: 190
})


In [ ]:
print(all_results["label"])
print(all_results["response"])

[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1]
["###Label: 0\n ###Prediction:  'YES': ['ACCESS_FINE_LOCATION: fine location (GPS)', 'ACCESS_FINE_LOCAL_WITH_GPS: fine location (GPS) and precise location (GPS)', 'ACCESS_NETWORK_STATE: view network connections', 'ACCESS_COARSE_LOCATION: approximate location (network-based)', 'ACCESS_WIFI_STATE: view Wi-Fi connections', 'READ", "###Label: 0\n ###Prediction:  NO\n   ### Context:\n   Permissions: ['ACCESS_NETWORK_STATE: view ne